In [2]:
import numpy as np
import matplotlib.pyplot as plt

from hawkes import UnivariateHP, MultivariateHP
from plot_hawkes import *

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Simulate a univariate HP

In [3]:
# Declare HP constants
mu = 0.1
alpha = 0.5
beta = 1

# Warning if HP parameters will make HP explode
if alpha >= beta**2:  # branching ratio = ||h||L1 = alpha/beta^2
    print("Need alpha < beta^2 for the branching ratio to be < 1")

# Define 2nd order B-spline kernel
def kernel(t, alpha=alpha, beta=beta):
    return alpha * t * np.exp(-beta*t) * (t >= 0)

# Define univariate HP
hp = UnivariateHP(mu=mu, h=kernel)

In [4]:
# Plot kernel
timevect = np.linspace(0, 10, 100)
plt.figure()
plt.plot(timevect, kernel(timevect))
plt.grid()

In [5]:
# Simulate and plot HP
T = 250  # simulation time
P = hp.simulate_univariate_hp(T=T)

In [6]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

plot_counting_process(P=P, M=1, axe=axes[0, 0])
plot_intensity(P=P, T=T, mu=mu, h=kernel, axe=axes[0, 1])
plot_histogram(P=P, T=T, axe=axes[1, 0])

### Simulate a multivariate HP

In [4]:
# Declare HP constants
M = 2  # number of variables/neurons
mu = np.array([0.1, 0.1])
alpha = np.array([[0.3, 0.2], 
                  [0.2, 0.3]])
beta = 1

# Warning if HP parameters will make HP explode
_, S, _ = np.linalg.svd(alpha)  # stability is determined by matrix H, H_ij = ||hij||_L1
spectralRadius = np.max(S)
if np.max(S) >= 1:
    print(f"Spectral radius of excitation matrix is {spectralRadius} > 1, HP is unstable.")
else:
    print(f"Spectral radius is {spectralRadius} < 1, HP is stable.")

# Define 2nd order B-spline kernels
kernels = np.empty((M, M), dtype=object)

for (i, j), alpha_ij in np.ndenumerate(alpha):
    def hij(t, alpha=alpha_ij, beta=beta):
        return alpha * t * np.exp(-beta*t) * (t >= 0)
    kernels[i, j] = hij

# Define multivariate HP
multihp = MultivariateHP(M=M, mu=mu, h=kernels)

In [5]:
# Plot kernels
timevect = np.linspace(0, 10, 100)
fig, axes = plt.subplots(M, M, figsize=(12, 8))

for (i, j), hij in np.ndenumerate(kernels):
    axes[i, j].plot(timevect, hij(timevect), label=f'h_{i}{j} with alpha={alpha[i,j]}')
    axes[i, j].grid()
    axes[i, j].legend()

In [6]:
# Simulate and plot HP
T = 250  # simulation time
P = multihp.simulate_multivariate_hp(T=T)

In [7]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

plot_counting_process(P, M=M, axe=axes[0, 0])
plot_intensity_multivariate(P=P, i=0, T=T, mu=mu, h=kernels, axe=axes[0, 1])
plot_intensity_multivariate(P=P, i=1, T=T, mu=mu, h=kernels, axe=axes[1, 1])
